In [1]:
import collections
import warnings
import os

import nengo
import nengo_dl
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nengo_loihi

# ignore NengoDL warning about no GPU
warnings.filterwarnings("ignore", message="No GPU", module="nengo_dl")

# The results in this notebook should be reproducible across many random seeds.
# However, some seed values may cause problems, particularly in the `to-spikes` layer
# where poor initialization can result in no information being sent to the chip. We set
# the seed to ensure that good results are reproducible without having to re-train.
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
tf.__version__, nengo_dl.__version__, nengo_loihi.__version__

('2.2.0', '3.4.0', '1.0.0')

In [3]:
# load mnist dataset
(train_images, train_labels), (
    test_images,
    test_labels,
) = tf.keras.datasets.mnist.load_data()

# flatten images and add time dimension
train_images = train_images.reshape((train_images.shape[0], 1, -1))
train_labels = train_labels.reshape((train_labels.shape[0], 1, -1))
test_images = test_images.reshape((test_images.shape[0], 1, -1))
test_labels = test_labels.reshape((test_labels.shape[0], 1, -1))

In [4]:
inp = tf.keras.Input(shape=(28, 28, 1), name="input")


#################### OFF-CHIP INPUT LAYER ###########################
# transform input signal to spikes using trainable 1x1 convolutional layer
to_spikes_layer = tf.keras.layers.Conv2D(
    filters=3,  # 3 neurons per pixel
    kernel_size=1,
    strides=1,
    activation=tf.nn.relu,
    use_bias=False,
    name="to-spikes",
)
to_spikes = to_spikes_layer(inp)

################################ ON-CHIP LAYERS ##################################
# on-chip convolutional layers
conv0_layer = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
    use_bias=False,
    name="conv0",
)
conv0 = conv0_layer(to_spikes)

conv1_layer = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
    use_bias=False,
    name="conv1",
)
conv1 = conv1_layer(conv0)

flatten = tf.keras.layers.Flatten(name="flatten")(conv1)
dense0_layer = tf.keras.layers.Dense(units=100, activation=tf.nn.relu, name="dense0")
dense0 = dense0_layer(flatten)

############################ OFF-CHIP OUTPUT LAYER #########################
# since this final output layer has no activation function,
# it will be converted to a `nengo.Node` and run off-chip
dense1 = tf.keras.layers.Dense(units=10, name="dense1")(dense0)

model = tf.keras.Model(inputs=inp, outputs=dense1)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
to-spikes (Conv2D)           (None, 28, 28, 3)         3         
_________________________________________________________________
conv0 (Conv2D)               (None, 13, 13, 32)        864       
_________________________________________________________________
conv1 (Conv2D)               (None, 6, 6, 64)          18432     
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense0 (Dense)               (None, 100)               230500    
_________________________________________________________________
dense1 (Dense)               (None, 10)                1010  

In [5]:
pres_time = 0.03  # how long to present each input, in seconds
n_test = 5  # how many images to test

# convert the keras model to a nengo network
nengo_converter = nengo_dl.Converter(
    model,
    scale_firing_rates=400,
    swap_activations={tf.nn.relu: nengo_loihi.neurons.LoihiSpikingRectifiedLinear()},
    synapse=0.005,
)
net = nengo_converter.net

# get input/output objects
nengo_input = nengo_converter.inputs[inp]
nengo_output = nengo_converter.outputs[dense1]

In [6]:
# build network, load in trained weights, save to network
with nengo_dl.Simulator(net) as nengo_sim:
    nengo_sim.load_params("keras_to_loihi_loihineuron_params_32_64")
    nengo_sim.freeze_params(net)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


In [7]:
with net:
    nengo_input.output = nengo.processes.PresentInput(
        test_images, presentation_time=pres_time
    )

with net:
    nengo_loihi.add_params(net)  # allow on_chip to be set
    net.config[nengo_converter.layers[to_spikes].ensemble].on_chip = False

In [10]:
# Set the Block Shapes on Loihi Cores. 
# (Nengo Loihi allocates one block per neurocore. Src: https://forum.nengo.ai/t/how-many-blocks-per-loihi-chip/1732)
# Ensembles can be distributed across one or more blocks => Ensembles are mapped to one or more neurocores.
# This implies that models with many small ensembles will fill up the chip more quickly (due to core segmentation).
#
# Therefore have Ensembles which can be split in groups/blocks such that they max out each core. E.g. an Ensemble 
# with 1025 neurons gets split across two cores, but would be using about half the neurons on each, whereas an 
# Ensemble with 2048 neurons would also take two cores and be using all neurons on each (1 neurocore has 1024 neurons).
# Above assumes that Ensembles don't need to be split across more cores because they are maxing out other resources,
# like: axons or synapse memory.
#
# A Loihi neurocore has just over a million bits to store all synapse information (
# Src: https://forum.nengo.ai/t/trying-to-understand-network-restrictions/927). In NengoLoihi, 8 bits per synapse
# is used => 1000000 / 8 = 125K synapses can be mapped to a Loihi neurocore. For an all to all Dense connection, you
# cannot go much larger than 300 x 300 weights. 
with net:
    conv0_shape = conv0_layer.output_shape[1:]
    net.config[
        nengo_converter.layers[conv0].ensemble
    ].block_shape = nengo_loihi.BlockShape((16, 16, 4), conv0_shape)

    conv1_shape = conv1_layer.output_shape[1:]
    net.config[
        nengo_converter.layers[conv1].ensemble
    ].block_shape = nengo_loihi.BlockShape((8, 8, 16), conv1_shape)

    dense0_shape = dense0_layer.output_shape[1:]
    net.config[
        nengo_converter.layers[dense0].ensemble
    ].block_shape = nengo_loihi.BlockShape((50,), dense0_shape)

In [9]:
# build NengoLoihi Simulator and run network
with nengo_loihi.Simulator(net, target="sim") as loihi_sim:
    loihi_sim.run(n_test * pres_time)

    # get output (last timestep of each presentation period)
    pres_steps = int(round(pres_time / loihi_sim.dt))
    output = loihi_sim.data[nengo_output][pres_steps - 1 :: pres_steps]

    # compute the Loihi accuracy
    loihi_predictions = np.argmax(output, axis=-1)
    correct = 100 * np.mean(loihi_predictions == test_labels[:n_test, 0, 0])
    print("Loihi accuracy: %.2f%%" % correct)

Loihi accuracy: 100.00%
